## Data Lake House
- A Data Lakehouse is a hybrid data architecture that combines the flexibility and scalability of a data lake with the performance and governance features of a data warehouse. 
- It enables organizations to run BI, machine learning (ML), and real-time analytics on a single platform.

### features
- structured, semi-structured, and unstructured data
- ACID 
- Schema enforcement
- Time Travel & Data Versioning
- Batch & stream workloads
* The underlying storage layer is cloud object storage only

### Databricks uses Delta live tables
#### Snowflake uses ApacheIceberg or Aws athena
- with help of apache iceberg snowflake directly queries from data lake storage with out ingesting into snowflake tables


## Delta Lake
- stores data in parquet format and columnar 
- Delta Lake adds a transaction log (_delta_log) that tracks all changes to the data and ensures ACID compliance.
- since ACID schema enforcement
- time travel is available with transaction log various versions of data is saved. 

### Components
- Delta table - a table that stores all of the data.
- Delta log -  transaction log 
- Delta Cache - transaction cache which stores recent versions of data.

- Upserts are done via MERGE or INSERT INTO
- checkpoints are used for recovery
- snapshots for rollback 


In [ ]:
##full scan mode


In [ ]:
## incremental mode